In [1]:
import numpy as np
import pandas as pd 

После загрузги библиотеки я сразу посмотрела, какие столбы загружаются, и оставила нужные, дабы при дальнейших загрузках на это не обращать внимание. 

In [2]:
data_xl = pd.ExcelFile("test_task.xlsx")
data = data_xl.parse(data_xl.sheet_names[0])
data.drop('good (1)', axis= 1 , inplace= True )
data.head()

,area,cluster,cluster_name,keyword,count,x,y
0,eligibility,0.0,Кластер 0,several animated buried,1260,5.772342,12.564796257345005
1,eligibility,0.0,Кластер 0,singles unusual buyers,866,14.829280,7.8507285727125815
2,eligibility,0.0,Кластер 0,hawaiian directive,163,11.381856,3.8981370219558604
3,eligibility,0.0,Кластер 0,dynamics directly,1146,9.980149,6.281427914064545
4,eligibility,1.0,Кластер 1,decision surgeons montreal,823,3.283940,4.39674063521296


Прежде чем что-то делать, надо узнать есть ли пропуски. 

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 229 entries, 0 to 228
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   area          228 non-null    object 
 1   cluster       228 non-null    float64
 2   cluster_name  228 non-null    object 
 3   keyword       228 non-null    object 
 4   count         227 non-null    object 
 5   x             228 non-null    float64
 6   y             228 non-null    object 
dtypes: float64(2), object(5)
memory usage: 12.6+ KB


In [4]:
b = []
for i in data['y']:
    try:
        a = float(i)
    except:
        b.append(i)
print(b)        
data = data.drop(index = data.index[data['y'].isin(b)]) # удаляю не числовые строки 
data['y'] = data['y'].astype (float)

['0x414fe002']


In [5]:
data.isnull().sum()[lambda x: x>0]

area            1
cluster         1
cluster_name    1
keyword         1
count           2
x               1
y               1
dtype: int64

Выяснилось, что они есть. Но в странном соотношении. Надо бы посмотреть на эти пропуски. 

In [6]:
list_nan= data.index[data.isnull().any(axis=1)]
data.iloc[list_nan]

,area,cluster,cluster_name,keyword,count,x,y
179,housewives,3.0,Кластер 3,florida reserved,1347,7.503056,8.610352
194,dialog,2.0,Кластер 2,publicity mongolia,1839,0.619892,2.681503


Это оказалась целиком пустая строку, которую мы просто удалим. А вот с единичным значениме в инструкции нет ничего. Так бы по хорошему надо было б узнать, можно ли его заменить средним значением. Т.к. предполагаем, что тестовое мы делаем без вопросов, то эту строчку мы то же удаляем. 

In [7]:
data = data.drop(index = list_nan)

In [8]:
data.isnull().sum()[lambda x: x>0]

Series([], dtype: int64)

Все, пропусков у нас нет. 
В условии у нас указан порядок: area, cluster, cluster_name, keyword, x, y, count, color.

Значит перемещаем столбец count в конец. Так же создаем и заполняем столбец color по правилам, которые есть в задании. 

In [9]:
data = data[['area', 'cluster', 'cluster_name', 'keyword', 'x', 'y', 'count']]
data.head()

,area,cluster,cluster_name,keyword,x,y,count
0,eligibility,0.0,Кластер 0,several animated buried,5.772342,12.564796,1260
1,eligibility,0.0,Кластер 0,singles unusual buyers,14.829280,7.850729,866
2,eligibility,0.0,Кластер 0,hawaiian directive,11.381856,3.898137,163
3,eligibility,0.0,Кластер 0,dynamics directly,9.980149,6.281428,1146
4,eligibility,1.0,Кластер 1,decision surgeons montreal,3.283940,4.396741,823


In [10]:
data['color'] = pd.Series()
data.head()

C:\Users\juiwa\AppData\Local\Temp\ipykernel_8860\1155315422.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  data['color'] = pd.Series()


,area,cluster,cluster_name,keyword,x,y,count,color
0,eligibility,0.0,Кластер 0,several animated buried,5.772342,12.564796,1260,NaN
1,eligibility,0.0,Кластер 0,singles unusual buyers,14.829280,7.850729,866,NaN
2,eligibility,0.0,Кластер 0,hawaiian directive,11.381856,3.898137,163,NaN
3,eligibility,0.0,Кластер 0,dynamics directly,9.980149,6.281428,1146,NaN
4,eligibility,1.0,Кластер 1,decision surgeons montreal,3.283940,4.396741,823,NaN


Цвет задается каждому словосочетанию согласно следующими правилам:
* внутри одной области цвета словосочетаний в одном кластере должны быть одинаковые, в разных - отличаться (например, у "Кластер 1" все слова будут окрашены в красный, у "Кластер 2" - в зеленый и т.д.)
* цвета кластеров в разных областях могут повторяться
* цвета кластеров в разных областях с разным номером не имеют никакой связи (у одной области [area] слова из "Кластер 1" могут быть красного цвета, в другой области у слов из "Кластер 1" может быть другой цвет)

По данному условию мы можем найти какое количество обозначений кластеров у нас есть - это и будет колличество наших цветов.  


In [11]:
data['cluster'].value_counts()

1.0    62
0.0    60
2.0    59
3.0    45
Name: cluster, dtype: int64

Значит будет 4 цвета. Т.к. их мало и не было сказано, как именно их надо задавать, то я задам их в ручную. В ином случае разумно былоб сделать генерацию цвета с привязкой к номеру. Т.е. цвета обозначать номерами. 

In [12]:
dict_color = {0: 'red',
              1:'blue',
              2: 'green',
              3: 'yellow'}

In [13]:
for key, value in dict_color.items():
    data['color'] = np.where((data.cluster == key ), value, data.color)

In [14]:
data.head()

,area,cluster,cluster_name,keyword,x,y,count,color
0,eligibility,0.0,Кластер 0,several animated buried,5.772342,12.564796,1260,red
1,eligibility,0.0,Кластер 0,singles unusual buyers,14.829280,7.850729,866,red
2,eligibility,0.0,Кластер 0,hawaiian directive,11.381856,3.898137,163,red
3,eligibility,0.0,Кластер 0,dynamics directly,9.980149,6.281428,1146,red
4,eligibility,1.0,Кластер 1,decision surgeons montreal,3.283940,4.396741,823,blue


Надо так же проверить, что бы небыло одинаковых словосочетаний в одной области. В условии не сказано, что с ними надо делать. Поэтому будем объединять в кластер так:
* Если колличество в одном класторе больше - соединяем в него
* Если колличество одинаково - объединяем произвольно

Вот тут у меня начились проблемы с типом, которые я не нашла во время поиска несостыковок в данных. Пришлось искать, где я проглядела не числовые значения. По итогу просто удалила их.  

In [15]:
b = []
for i in data['count']:
    if type(i)!=int: 
        b.append(i)  
data = data.drop(index = data.index[data['count'].isin(b)]) # удаляю не числовые строки    
data['count'] = data['count'].astype (int) # и преобразую все в числаы

In [16]:
list_area = data.area.unique()  

In [17]:
for i in list_area:
    df = data.loc[data['area'] == i]
    duplicateRows = df[df.duplicated(['keyword'])]
    if not duplicateRows.empty: # проверяем, что датасет не пустой
        list_dubl = duplicateRows['keyword'].tolist() # лист со словами-дублями
        for dubl in list_dubl:
            list_dubl_ind = df.index[ df['keyword'] == dubl].tolist() # индексы, где лежат дубликаты
            ind = df[df['keyword'] == dubl]['count'].idxmax() # индекс, куда в и итоге мы объединяем             
            data.at[ind, 'count'] = df[df['keyword'] == dubl]['count'].sum()

            list_dubl_ind.remove(ind)
            data = data.drop(index = list_dubl_ind) # удаляю дубли

In [18]:
data.to_excel('Processed_table.xlsx', index=False)